In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import datetime
import boto3
from io import StringIO
import gzip

In [ ]:
# Create an STS client
sts_client = boto3.client('sts')

# Assume the 'dataexportrole' role with an external ID
assumed_role = sts_client.assume_role(
    RoleArn="",
    RoleSessionName="AssumeRoleSession1",
    ExternalId="VsCode"
)

# Extract the temporary credentials
credentials = assumed_role['Credentials']

# Create a new session with the assumed role's temporary credentials
session = boto3.Session(
    aws_access_key_id=credentials['AccessKeyId'],
    aws_secret_access_key=credentials['SecretAccessKey'],
    aws_session_token=credentials['SessionToken'],
    region_name='us-east-1'
)

# Use the session to create an S3 client
s3_client = session.client('s3')

bucket_name = ''
data_export_file_path = ''

# Access the S3 object
response = s3_client.get_object(Bucket=bucket_name, Key=data_export_file_path)
gzipped_content = response['Body'].read()

# Decompress and read the content
decompressed_content = gzip.decompress(gzipped_content)
data_string = StringIO(decompressed_content.decode('utf-8'))
data = pd.read_csv(data_string, index_col=0)


In [ ]:
data['line_item_usage_end_date'] = data['line_item_usage_end_date'].str.replace('T0', 'T', regex=False)
data['line_item_usage_end_date'] = pd.to_datetime(data['line_item_usage_end_date'])
data.set_index('line_item_usage_end_date', inplace=True)

In [ ]:
filtered_data = data[data['line_item_operation'] == 'Rules']
hourly_data = filtered_data['pricing_public_on_demand_cost'].resample('h').sum()
model = SARIMAX(hourly_data, order=(1,1,1), seasonal_order=(1,1,1,12))
model_fit = model.fit()

n_hours = 24 * 31
forecast = model_fit.forecast(steps=n_hours)

forecast_daily = np.array(forecast).reshape(-1, 24)
daily_costs = forecast_daily.sum(axis=1)

In [ ]:
print('Hourly costs\n', forecast)

In [ ]:
print('\nDaily costs')
for i, cost in enumerate(daily_costs, start=1):
    print(f'Daily cost for day {i}: ${cost}')

In [ ]:
total_monthly_cost = sum(daily_costs)

print(f'\nTotal monthly cost: ${total_monthly_cost}')